<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:LightSlateGray;
           font-size:150%;
           text-align:center;
            letter-spacing:0.5px"> <a id='4'></a>
    
    Dashboard
</div>

<div style="color:black;
           display:fill;
           border-radius:5px;
           background-color:Beige;
           font-size:110%;
           letter-spacing:0.5px">
<p style="padding: 10px; color:black; font-weight:bold;">

Below we will be implementing the dashboard which will provide the recommendations of the movie upon selection and also personalised according to the user.
    
As of now we have implemented 2 Algorithms, KNN and Content Based on the Dashboard.
<strong>Please run this script only when you completed the run of Movie Recommendation.ipynb file</strong>
</p>
</div>

In [1]:
import dash
import pandas as pd
from dash import dcc, html, Input, Output
import requests
from dash.dependencies import Input, Output
import random

# Load movie data from a pickle file (replace with your data)
movie_dict = pd.read_pickle('movie_list.pkl')
movies = pd.DataFrame(movie_dict)
similarity = pd.read_pickle('cba_similarity.pkl')
movies_knn = pd.read_csv('movie_recommendations_knn.csv')
user_details = pd.read_csv('grouped_user_filtered_df.csv')
genres_list = pd.read_pickle('unique_genres.pkl')
cast_list = pd.read_pickle('unique_cast.pkl')
user_semantics = pd.read_csv('user_semantics.csv')

app = dash.Dash(__name__, external_stylesheets=["style.css"])

# Convert the 'genres' column to strings and handle NaN values
movies['genres'] = movies['genres'].astype(str)

# Create a list of movie name options for the dropdown from the DataFrame
movie_name_options = [{'label': movie, 'value': movie} for movie in movies['title']]
cast_options = [{'label': cast, 'value': cast} for cast in cast_list]

# Define the layout with dropdowns for movie title and user_id selection

app.layout = html.Div([
    html.H1("Movie Recommendations Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            dcc.Dropdown(
                id="user-id-dropdown",
                options=[{'label': str(userId), 'value': userId} for userId in user_details['userId']],
                placeholder="Select a user_id",
                style={'width': '60%', 'margin-right': '7px'}
            ),
            dcc.Dropdown(
                id="movie-title-dropdown",
                options=movie_name_options,
                placeholder="Select a movie you watched or liked",
                style={'width': '60%', 'margin-right': '7px'}
            ),
            dcc.Dropdown(
                id="cast-dropdown",
                options=cast_options,
                multi=True,
                placeholder="Select cast",
                style={'width': '60%', 'margin-right': '7px'}
            ),
        ], style={'display': 'flex'}),

        html.Div([
            dcc.Checklist(
                id="genre-checklist",
                options=[{'label': genre, 'value': genre} for genre in genres_list],
                value=[],  # Set default values to an empty list (unselected)
                inline=True,  # Display options horizontally
                style={'width': '60%', 'margin-top': '10px'}  # Adjust width as needed
            ),
            html.Button("Get Recommendations", id="get-recommendation-button", n_clicks=0, className='btn-primary',
                        style={'margin-top': '10px'}),
            html.Button("Unselect All", id="unselect-all-button", n_clicks=0, className='btn-primary',
                        style={'margin-top': '10px', 'margin-left': '10px'})
        ], style={'display': 'flex'}),
    ]),

    html.Div([
        html.H2("Movie Recommendations", style={'color': 'Blue', 'font-style': 'italic', 'font-weight': 'bold'}),
        html.Div(id="scenario-explanation", className="row"),
    ]),

    html.Div([
        html.Div(id="recommended-movies-cba", className="row"),
    ]),

    html.Div([
        html.Div(id="recommended-movies-knn", className="row"),
    ]),

    html.Div([
        html.Div(id="recommend_genre_and_cast", className="row"),
    ]),

    html.Div([
        html.Div(id="recommend_user_semantics", className="row"),
    ])
])


@app.callback(
    Output("recommended-movies-cba", "children"),
    Output("recommended-movies-knn", "children"),
    Output("recommend_genre_and_cast", "children"),
    Output("recommend_user_semantics", "children"),
    Input("get-recommendation-button", "n_clicks"),
    Input("movie-title-dropdown", "value"),
    Input("user-id-dropdown", "value"),
    Input("unselect-all-button", "n_clicks"),
    Input("genre-checklist", "value"),
    Input("cast-dropdown", "value"),
    prevent_initial_call=True
)
def get_recommendations(n_clicks, selected_movie, selected_user_id, unselect_all_clicks, selected_genres,
                        selected_cast):
    common_movie_names_out = []
    if n_clicks > 0 and (selected_movie or selected_user_id or selected_genres or selected_cast):
        if not user_details.empty and selected_user_id in user_details['userId'].values:
            user_watched_movies_str = user_details[user_details['userId'] == selected_user_id]['movieId'].values[0]
            user_watched_movie_ids = [int(movie_id.strip()) for movie_id in
                                      user_watched_movies_str.strip('[]').split(',')]
            user_watched_movies = [movies[movies['movieId'] == movie_id]['title'].values[0] for movie_id in
                                   user_watched_movie_ids]
        else:
            user_watched_movies = []

        if selected_user_id is not None:
            # Step 1: Get unique watched movie IDs from user_semantics for the selected user
            watched_movie_ids_out = set(
                user_semantics[user_semantics['userId'] == selected_user_id]['movieId'].tolist())

            # Step 2: Find movie IDs that match with selected genres from movies
            matching_movie_ids_out = set(movies[
                                             movies['genres'].apply(
                                                 lambda genres: any(genre in genres for genre in selected_genres))
                                         ]['movieId'].tolist())

            # Step 3: Get the intersection of watched_movie_ids and matching_movie_ids
            common_movie_ids_out = watched_movie_ids_out.intersection(matching_movie_ids_out)

            # Step 4: Sort common_movie_ids by the sum of ratings in user_semantics
            common_movie_ids_check = sorted(common_movie_ids_out, key=lambda x:
            user_semantics[(user_semantics['userId'] == selected_user_id) & (user_semantics['movieId'] == x)][
                'rating'].sum(), reverse=True)[:8]

            # Step 5: Get the corresponding movie names
            common_movie_names_out = [movies[movies['movieId'] == movie_id]['title'].values[0] for movie_id in
                                      common_movie_ids_check]
        # Get Content-Based recommendations
        recommended_movies_cba = recommend_cba(selected_movie, user_watched_movies, top_n=20)

        # Get KNN recommendations
        recommended_movies_knn = recommend_knn(selected_movie, user_watched_movies)

        # Get Genre and Cast recommendations
        recommended_movies_genre_cast = recommend_genre_and_cast(selected_genres, selected_user_id, selected_cast,
                                                                 user_watched_movies, user_semantics,
                                                                 top_n=20)

        recommend_movies_user_semantics, explanations_user_semantics = recommend_user_seman(selected_user_id,
                                                                                            user_watched_movies)

        recommended_movie_list_cba = []
        recommended_movie_list_knn = []
        recommended_movies_list_genre_cast = []
        recommended_movies_list_user_semantics = []

        if recommended_movies_cba:
            for movie_name in recommended_movies_cba:
                # Create recommendation columns for Content-Based recommendations
                poster_url = fetch_poster(get_tmdbId(movie_name))
                if poster_url:
                    poster_image = html.Img(src=poster_url, style={'height': '200px', 'margin-right': '10px'})
                    movie_name_label = html.P("Recommended Movie: " + movie_name,
                                              style={'color': 'green', 'font-weight': 'bold', 'font-style': 'italic'},
                                              className='movie-name')
                    genres = get_genres(movie_name)
                    genres_label = html.P("Genres: " + genres, className='genres')
                    cast_info, director_info = get_cast_and_director(movie_name)
                    cast_label = html.P("Cast: " + cast_info, className='cast')
                    director_label = html.P("Director: " + director_info, className='director')
                    explanation_label = html.P(
                        f"Explanation: Based on your selection of '{selected_movie}', we recommend '{movie_name}' due to several factors. Firstly, there is a notable similarity in genre, which is evident from the genres listed above. Additionally, the recommendation is influenced by shared characteristics and storyline features between the two movies. Moreover, '{selected_movie}' and '{movie_name}' share some common cast and crew members, further contributing to this recommendation. This recommendation is generated using the Content-Based Algorithm.",
                        style={'color': 'brown', 'font-style': 'italic', 'font-weight': 'bold'},
                        className='explanation')
                    recommendation_column = html.Div([
                        poster_image,
                        html.Div([
                            movie_name_label,
                            genres_label,
                            cast_label,
                            director_label,
                            explanation_label
                        ], className='recommendation-column')
                    ], className='recommendation-column')
                    recommended_movie_list_cba.append(recommendation_column)

        if recommended_movies_knn:
            for movie_name in recommended_movies_knn:
                # Create recommendation columns for KNN recommendations
                poster_url = fetch_poster(get_tmdbId(movie_name))
                if poster_url:
                    poster_image = html.Img(src=poster_url, style={'height': '200px', 'margin-right': '10px'})
                    movie_name_label = html.P("Recommended Movie: " + movie_name,
                                              style={'color': 'green', 'font-weight': 'bold', 'font-style': 'italic'},
                                              className='movie-name')
                    genres = get_genres(movie_name)
                    genres_label = html.P("Genres: " + genres, className='genres')
                    cast_info, director_info = get_cast_and_director(movie_name)
                    cast_label = html.P("Cast: " + cast_info, className='cast')
                    director_label = html.P("Director: " + director_info, className='director')
                    explanation_label = html.P(
                        f"Explanation: Regarding your selection of '{selected_movie}', we recommend '{movie_name}' because it shares a common set of ratings with viewers who enjoyed '{selected_movie}'. This recommendation is based on the idea that users who liked '{selected_movie}' and rated it similarly are likely to appreciate '{movie_name}' as well. K-Nearest Neighbors (KNN) with Matrix Factorization is the method used to make this recommendation, taking into account the alignment of user ratings.",
                        style={'color': 'brown', 'font-style': 'italic', 'font-weight': 'bold'},
                        className='explanation')
                    recommendation_column = html.Div([
                        poster_image,
                        html.Div([
                            movie_name_label,
                            genres_label,
                            cast_label,
                            director_label,
                            explanation_label
                        ], className='recommendation-column')
                    ], className='recommendation-column')
                    recommended_movie_list_knn.append(recommendation_column)

        if recommended_movies_genre_cast:
            for movie_name in recommended_movies_genre_cast:
                # Create recommendation columns for Genre_cast recommendations
                poster_url = fetch_poster(get_tmdbId(movie_name))
                if poster_url:
                    poster_image = html.Img(src=poster_url, style={'height': '200px', 'margin-right': '10px'})
                    movie_name_label = html.P("Recommended Movie: " + movie_name,
                                              style={'color': 'green', 'font-weight': 'bold', 'font-style': 'italic'},
                                              className='movie-name')
                    genres = get_genres(movie_name)
                    genres_label = html.P("Genres: " + genres, className='genres')
                    cast_info, director_info = get_cast_and_director(movie_name)
                    cast_label = html.P("Cast: " + cast_info, className='cast')
                    director_label = html.P("Director: " + director_info, className='director')

                    # Check if userId is selected to modify the explanation
                    if selected_user_id and common_movie_names_out:
                        random_name = random.choice(common_movie_names_out)
                        explanation_label = html.P(
                            f"Explanation: The movie, '{movie_name}', has been recommended for you as from previous intercations with you we can see that you liked/highly rated/reviewed '{random_name}' which belongs to the selected genres above and the '{movie_name}' and '{random_name}' have lot of similarities which helped us in recommending this movie to you as you have not watched this movie yet.",
                            style={'color': 'brown', 'font-style': 'italic', 'font-weight': 'bold'},
                            className='explanation')
                    else:
                        if selected_user_id is not None:
                            explanation_label = html.P(
                                f"Explanation: The movie, '{movie_name}', has been recommended to user with ID '{selected_user_id}' as they may have selected cast(s) or genre(s) that match with the movie characteristics and also have liked/highly rated/reviewed movies similar to '{movie_name}'. If you want a combined result of genre and cast, please enter the details in both sections.",
                                style={'color': 'brown', 'font-style': 'italic', 'font-weight': 'bold'},
                                className='explanation')
                        else:
                            explanation_label = html.P(
                                f"Explanation: The movie, '{movie_name}', has been recommended to you as you may have selected cast(s) or genre(s) that match with the movie characteristics. If you want a combined result of genre and cast, please enter the details in both sections.",
                                style={'color': 'brown', 'font-style': 'italic', 'font-weight': 'bold'},
                                className='explanation')

                    recommendation_column = html.Div([
                        poster_image,
                        html.Div([
                            movie_name_label,
                            genres_label,
                            cast_label,
                            director_label,
                            explanation_label
                        ], className='recommendation-column')
                    ], className='recommendation-column')
                    recommended_movies_list_genre_cast.append(recommendation_column)

        if recommend_movies_user_semantics:
            for movie_name, explanation_user_semantics in zip(recommend_movies_user_semantics,
                                                              explanations_user_semantics):
                # Create recommendation columns for user_semantics recommendations
                poster_url = fetch_poster(get_tmdbId(movie_name))
                if poster_url:
                    poster_image = html.Img(src=poster_url, style={'height': '200px', 'margin-right': '10px'})
                    movie_name_label = html.P("Recommended Movie: " + movie_name,
                                              style={'color': 'green', 'font-weight': 'bold', 'font-style': 'italic'},
                                              className='movie-name')
                    genres = get_genres(movie_name)
                    genres_label = html.P("Genres: " + genres, className='genres')
                    cast_info, director_info = get_cast_and_director(movie_name)
                    cast_label = html.P("Cast: " + cast_info, className='cast')
                    director_label = html.P("Director: " + director_info, className='director')
                    explanation_label = html.P(
                        explanation_user_semantics,
                        style={'color': 'brown', 'font-style': 'italic', 'font-weight': 'bold'},
                        className='explanation')
                    recommendation_column = html.Div([
                        poster_image,
                        html.Div([
                            movie_name_label,
                            genres_label,
                            cast_label,
                            director_label,
                            explanation_label
                        ], className='recommendation-column')
                    ], className='recommendation-column')
                    recommended_movies_list_user_semantics.append(recommendation_column)

        return recommended_movie_list_cba, recommended_movie_list_knn, recommended_movies_list_genre_cast, recommended_movies_list_user_semantics

    # If no recommendations, return placeholders
    return [], [], [], []


def recommend_genre_and_cast(selected_genres, selected_user_id, selected_cast, user_watched_movies, user_semantics,
                             top_n=20):
    # Filter movies based on selected genres
    filtered_movies_genre = movies[
        movies['genres'].apply(lambda genres: any(genre in genres for genre in selected_genres))]

    if not selected_cast:
        # Case 1: Only genres selected, recommend movies with genre
        if filtered_movies_genre.empty:
            return []
        selected_movies_rows = filtered_movies_genre.sample(min(top_n, len(filtered_movies_genre)))
        recommended_movies_genre_cast = selected_movies_rows['title'].tolist()
        movies_list = filter_watched_movies(selected_genres, recommended_movies_genre_cast, selected_user_id,
                                            user_semantics,
                                            user_watched_movies)
        return movies_list

    # Case 2: Only cast selected, recommend movies with the selected cast
    if not selected_genres:
        filtered_movies_cast = movies[movies['cast'].apply(
            lambda cast: any(actor in cast.split(', ') for actor in selected_cast) if cast and isinstance(cast,
                                                                                                          str) else False)]

        if filtered_movies_cast.empty:
            return []
        selected_movies_rows = filtered_movies_cast.sample(min(top_n, len(filtered_movies_cast)))
        recommended_movies_genre_cast = selected_movies_rows['title'].tolist()
        return filter_watched_movies_cast(recommended_movies_genre_cast, selected_user_id, user_watched_movies)

    # Case 3: Both genre and cast selected, recommend movies with cast and genre
    filtered_movies = filtered_movies_genre[filtered_movies_genre['cast'].apply(
        lambda cast: any(actor in cast.split(', ') for actor in selected_cast) if cast and isinstance(cast,
                                                                                                      str) else False)]

    if filtered_movies.empty:
        return []

    selected_movies_rows = filtered_movies.sample(min(top_n, len(filtered_movies)))
    recommended_movies_genre_cast = selected_movies_rows['title'].tolist()
    movies_list = filter_watched_movies(selected_genres, recommended_movies_genre_cast, selected_user_id,
                                        user_semantics,
                                        user_watched_movies)
    return movies_list


def filter_watched_movies_cast(recommended_movies, selected_user_id, user_watched_movies):
    return [movie for movie in recommended_movies if movie not in user_watched_movies]


def filter_watched_movies(selected_genres, recommended_movies, selected_user_id, user_semantics, user_watched_movies):
    common_movie_names = []
    if selected_user_id is not None:
        # Step 1: Get unique watched movie IDs from user_semantics for the selected user
        watched_movie_ids = set(user_semantics[user_semantics['userId'] == selected_user_id]['movieId'].tolist())

        # Step 2: Find movie IDs that match with selected genres from movies
        matching_movie_ids = set(movies[
                                     movies['genres'].apply(
                                         lambda genres: any(genre in genres for genre in selected_genres))
                                 ]['movieId'].tolist())

        # Step 3: Get the intersection of watched_movie_ids and matching_movie_ids
        common_movie_ids = watched_movie_ids.intersection(matching_movie_ids)

        # Step 4: Sort common_movie_ids by the sum of ratings in user_semantics
        common_movie_ids = sorted(common_movie_ids, key=lambda x:
        user_semantics[(user_semantics['userId'] == selected_user_id) & (user_semantics['movieId'] == x)][
            'rating'].sum(), reverse=True)[:8]

        # Step 5: Get the corresponding movie names
        common_movie_names = [movies[movies['movieId'] == movie_id]['title'].values[0] for movie_id in common_movie_ids]

        # Step 6: Get CBA recommendations for each common movie (5 recommendations for each movie)
        cba_recommended_movies = []
        for movie_name in common_movie_names:
            recommendations_cba = recommend_cba(movie_name, user_watched_movies, top_n=3)
            cba_recommended_movies.extend(recommendations_cba)

        # Step 7: Compare genres of recommended movies with selected genres
        filtered_movies = [
            movie for movie in cba_recommended_movies
            if any(genre in movies[movies['title'] == movie]['genres'].values[0] for genre in selected_genres)
        ]
    else:
        filtered_movies = [movie for movie in recommended_movies if movie not in user_watched_movies]
    return filtered_movies


# Content-Based Recommendation logic function
def recommend_cba(movie, user_watched_movies, top_n=1):
    if movie not in movies['title'].values:
        return []
    movie_index = movies[movies['title'] == movie].index[0]
    dis = similarity[movie_index]
    movie_list = sorted(list(enumerate(dis)), reverse=True, key=lambda x: x[1])[1:(top_n + 1)]
    recommended_movies = []
    for i in movie_list:
        recommended_movie = movies.iloc[i[0]].title
        if recommended_movie not in user_watched_movies:
            recommended_movies.append(recommended_movie)
    return recommended_movies


# KNN Recommendation logic function
def recommend_knn(selected_movie, user_watched_movies):
    recommended_movie_list_knn = []
    # Find the row where MovieName matches the selected movie in movies_knn DataFrame
    movie_row = movies_knn[movies_knn['MovieName'] == selected_movie]
    if not movie_row.empty:
        # Split the Recommendations string and add to the KNN recommendations list
        recommendations_str = movie_row.iloc[0]['Recommendations']
        recommendations_list = [movie.strip('""') for movie in recommendations_str.split('", "')]

        for movie_name in recommendations_list:
            if movie_name not in user_watched_movies:
                recommended_movie_list_knn.append(movie_name)

    return recommended_movie_list_knn


def recommend_user_seman(selected_user_id, user_watched_movies):
    # Filter user_semantics data for the selected user
    user_data = user_semantics[user_semantics['userId'] == selected_user_id]

    if user_data.empty:
        return [], []

    if len(user_watched_movies) > 10:
        liked_movie_ids = user_data[user_data['liked(Yes_No)'] == 'Yes']['movieId'].sample(n=8).tolist()
        high_rated_movie_ids = user_data[(user_data['rating'] == 4) | (user_data['rating'] == 4.5) | (user_data['rating'] == 5)]['movieId'].tolist()
        high_rated_selected_movie_ids = random.sample(high_rated_movie_ids, min(8, len(high_rated_movie_ids)))
        high_reviewed_movie_ids = user_data[user_data['reviews'].str.contains('Excellent|Good')]['movieId'].sample(n=8).tolist()
    else:
        liked_movie_ids = user_data[user_data['liked(Yes_No)'] == 'Yes']['movieId'].tolist()
        high_rated_movie_ids = user_data[(user_data['rating'] == 4) | (user_data['rating'] == 4.5) | (user_data['rating'] == 5)]['movieId'].tolist()
        high_reviewed_movie_ids = user_data[user_data['reviews'].str.contains('Excellent|Good')]['movieId'].tolist()

    # Initialize lists to store recommendations and explanations
    all_recommended_movies = []
    all_explanations = []

    # Iterate over each liked movie
    for liked_movie_id in liked_movie_ids:
        # Check if movieId exists in the movies DataFrame
        if liked_movie_id in movies['movieId'].values:
            # Get the movie name corresponding to the liked_movie_id
            liked_movie_name = movies.loc[movies['movieId'] == liked_movie_id, 'title'].iloc[0]
            cast_info, director_info = get_cast_and_director(liked_movie_name)
            # Call recommend_cba to get recommendations for the liked movie
            recommended_movies_liked = recommend_cba(liked_movie_name, user_watched_movies, top_n=1)

            # Generate explanations for each recommendation
            explanations_likes = [
                f"Explanation: As you have previously liked the movie '{liked_movie_name}' starring '{cast_info}' and directed by '{director_info}', you get above movie as a recommendation as this movie have lots of similarity with genre, cast or other characteristics with the movie you liked.This movie is recommended to you using User Semantics and Content Based Algorithm."
                for _ in recommended_movies_liked
            ]

            # Append recommendations and explanations to the respective lists
            all_recommended_movies.extend(recommended_movies_liked)
            all_explanations.extend(explanations_likes)

    for high_rated_selected_movie_id in high_rated_selected_movie_ids:
        # Check if movieId exists in the movies DataFrame
        if high_rated_selected_movie_id in movies['movieId'].values:
            # Get the movie name corresponding to the liked_movie_id
            high_rated_selected_movie_name = \
                movies.loc[movies['movieId'] == high_rated_selected_movie_id, 'title'].iloc[0]
            cast_info, director_info = get_cast_and_director(high_rated_selected_movie_name)
            # Call recommend_cba to get recommendations for the liked movie
            recommended_movies_rated = recommend_cba(high_rated_selected_movie_name, user_watched_movies, top_n=1)

            # Generate explanations for each recommendation
            explanations_rated = [
                f"Explanation: As you have previously highly rated the movie '{high_rated_selected_movie_name}' starring '{cast_info}' and directed by '{director_info}' with a score of '4','4.5', or '5', you get  above movie as a recommendation as this movies have a lot of similarity with genre, cast or other characteristics with the highly rated movie by you.This movie is recommended to you using User Semantics and Content Based Algorithm."
                for _ in recommended_movies_rated
            ]

            # Append recommendations and explanations to the respective lists
            all_recommended_movies.extend(recommended_movies_rated)
            all_explanations.extend(explanations_rated)

    for high_reviewed_movie_id in high_reviewed_movie_ids:
        # Check if movieId exists in the movies DataFrame
        if high_reviewed_movie_id in movies['movieId'].values:
            # Get the movie name corresponding to the liked_movie_id
            high_reviewed_movie_name = movies.loc[movies['movieId'] == high_reviewed_movie_id, 'title'].iloc[0]
            cast_info, director_info = get_cast_and_director(high_reviewed_movie_name)
            # Call recommend_cba to get recommendations for the liked movie
            recommended_movies_reviewed = recommend_cba(high_reviewed_movie_name, user_watched_movies, top_n=1)

            # Generate explanations for each recommendation
            explanations_reviewed = [
                f"Explanation: As you have previously reviewed the movie '{high_reviewed_movie_name}' starring '{cast_info}' and directed by '{director_info}' as either 'Excellent' or 'Good' movie in the review comments, you get the above movie as a recommendation as both of the movies have a lot of similarity with genre, cast or other characteristics with the reviewed movie by you.This movie is recommended to you using User Semantics and Content Based Algorithm."
                for _ in recommended_movies_reviewed
            ]

            # Append recommendations and explanations to the respective lists
            all_recommended_movies.extend(recommended_movies_reviewed)
            all_explanations.extend(explanations_reviewed)

    return all_recommended_movies, all_explanations


# Function to fetch movie poster URL based on tmdbId using TMDb API
def fetch_poster(tmdbId):
    try:
        url = "https://api.themoviedb.org/3/movie/{}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US".format(
            tmdbId)
        data = requests.get(url)
        data = data.json()
        poster_path = data['poster_path']
        full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
        return full_path
    except Exception as e:
        # print("Error fetching poster URL:", str(e))
        return None


def get_genres(movie_name):
    genres = movies.loc[movies['title'] == movie_name, 'genres'].values[0]
    return genres


# Function to fetch cast and director information
def get_cast_and_director(movie_name):
    cast_info = str(movies.loc[movies['title'] == movie_name, 'cast'].values[0])
    director_info = str(movies.loc[movies['title'] == movie_name, 'director'].values[0])
    return cast_info, director_info


def get_tmdbId(movie_name):
    try:
        tmdbId = movies.loc[movies['title'] == movie_name, 'tmdbId'].values[0]
        return str(tmdbId)  # Convert to string for API request
    except IndexError:
        return None


def recommend_user_seman(selected_user_id, user_watched_movies):
    # Filter user_semantics data for the selected user
    user_data = user_semantics[user_semantics['userId'] == selected_user_id]

    if user_data.empty:
        return [], []

    if len(user_watched_movies) > 10:
        liked_movie_ids = user_data[user_data['liked(Yes_No)'] == 'Yes']['movieId'].sample(n=8).tolist()
        high_rated_movie_ids = user_data[(user_data['rating'] == 4) | (user_data['rating'] == 4.5) | (user_data['rating'] == 5)]['movieId'].tolist()
        high_rated_selected_movie_ids = random.sample(high_rated_movie_ids, min(8, len(high_rated_movie_ids)))
        high_reviewed_movie_ids = user_data[user_data['reviews'].str.contains('Excellent|Good')]['movieId'].sample(n=8).tolist()
    else:
        liked_movie_ids = user_data[user_data['liked(Yes_No)'] == 'Yes']['movieId'].tolist()
        high_rated_movie_ids = user_data[(user_data['rating'] == 4) | (user_data['rating'] == 4.5) | (user_data['rating'] == 5)]['movieId'].tolist()
        high_rated_selected_movie_ids = high_rated_movie_ids
        high_reviewed_movie_ids = user_data[user_data['reviews'].str.contains('Excellent|Good')]['movieId'].tolist()

    # Initialize lists to store recommendations and explanations
    all_recommended_movies = []
    all_explanations = []

    # Iterate over each liked movie
    for liked_movie_id in liked_movie_ids:
        # Check if movieId exists in the movies DataFrame
        if liked_movie_id in movies['movieId'].values:
            # Get the movie name corresponding to the liked_movie_id
            liked_movie_name = movies.loc[movies['movieId'] == liked_movie_id, 'title'].iloc[0]
            cast_info, director_info = get_cast_and_director(liked_movie_name)
            # Call recommend_cba to get recommendations for the liked movie
            recommended_movies_liked = recommend_cba(liked_movie_name, user_watched_movies, top_n=1)

            # Generate explanations for each recommendation
            explanations_likes = [
                f"Explanation: As you have previously liked the movie '{liked_movie_name}' starring '{cast_info}' and directed by '{director_info}', you get above movie as a recommendation as this movie have lots of similarity with genre, cast or other characteristics with the movie you liked.This movie is recommended to you using User Semantics and Content Based Algorithm."
                for _ in recommended_movies_liked
            ]

            # Append recommendations and explanations to the respective lists
            all_recommended_movies.extend(recommended_movies_liked)
            all_explanations.extend(explanations_likes)

    for high_rated_selected_movie_id in high_rated_selected_movie_ids:
        # Check if movieId exists in the movies DataFrame
        if high_rated_selected_movie_id in movies['movieId'].values:
            # Get the movie name corresponding to the liked_movie_id
            high_rated_selected_movie_name = \
                movies.loc[movies['movieId'] == high_rated_selected_movie_id, 'title'].iloc[0]
            cast_info, director_info = get_cast_and_director(high_rated_selected_movie_name)
            # Call recommend_cba to get recommendations for the liked movie
            recommended_movies_rated = recommend_cba(high_rated_selected_movie_name, user_watched_movies, top_n=1)

            # Generate explanations for each recommendation
            explanations_rated = [
                f"Explanation: As you have previously highly rated the movie '{high_rated_selected_movie_name}' starring '{cast_info}' and directed by '{director_info}' with a score of '4','4.5', or '5', you get  above movie as a recommendation as this movies have a lot of similarity with genre, cast or other characteristics with the highly rated movie by you.This movie is recommended to you using User Semantics and Content Based Algorithm."
                for _ in recommended_movies_rated
            ]

            # Append recommendations and explanations to the respective lists
            all_recommended_movies.extend(recommended_movies_rated)
            all_explanations.extend(explanations_rated)

    for high_reviewed_movie_id in high_reviewed_movie_ids:
        # Check if movieId exists in the movies DataFrame
        if high_reviewed_movie_id in movies['movieId'].values:
            # Get the movie name corresponding to the liked_movie_id
            high_reviewed_movie_name = movies.loc[movies['movieId'] == high_reviewed_movie_id, 'title'].iloc[0]
            cast_info, director_info = get_cast_and_director(high_reviewed_movie_name)
            # Call recommend_cba to get recommendations for the liked movie
            recommended_movies_reviewed = recommend_cba(high_reviewed_movie_name, user_watched_movies, top_n=1)

            # Generate explanations for each recommendation
            explanations_reviewed = [
                f"Explanation: As you have previously reviewed the movie '{high_reviewed_movie_name}' starring '{cast_info}' and directed by '{director_info}' as either 'Excellent' or 'Good' movie in the review comments, you get the above movie as a recommendation as both of the movies have a lot of similarity with genre, cast or other characteristics with the reviewed movie by you.This movie is recommended to you using User Semantics and Content Based Algorithm."
                for _ in recommended_movies_reviewed
            ]

            # Append recommendations and explanations to the respective lists
            all_recommended_movies.extend(recommended_movies_reviewed)
            all_explanations.extend(explanations_reviewed)

    return all_recommended_movies, all_explanations


# Unselect all callback function
@app.callback(
    Output("genre-checklist", "value"),
    Input("unselect-all-button", "n_clicks"),
)
def unselect_all(n_clicks):
    if n_clicks > 0:
        return []
    else:
        raise dash.exceptions.PreventUpdate


@app.callback(
    Output("scenario-explanation", "children"),
    [
        Input("user-id-dropdown", "value"),
        Input("genre-checklist", "value"),
        Input("cast-dropdown", "value"),
        Input("movie-title-dropdown", "value"),
    ]  # ,
    # prevent_initial_call=True
)
def update_scenario_explanation(selected_user_id, selected_genres, selected_cast, selected_movie):
    if not selected_user_id and not selected_genres and not selected_cast and not selected_movie:
        return [html.P(
            "Hi Movie Lover, please select your user id or, if you are new user, choose an option from above sections to get personalized recommendations.",
            style={'color': 'green', 'font-style': 'italic', 'font-weight': 'bold'})]

    if selected_user_id:
        explanation_text_1 = f"Hi user '{selected_user_id}'. Welcome back to Movie Recommendation Dashboard.\n"

        explanation_text = explanation_text_1 + "\n Click on 'Get Recommendations’.\n" \
                                                "\n Dashboard will provide you a personalized movie recommendations based on your previous watch history by taking into consideration likes, reviews, and ratings.\n" \
                                                "\nThe recommendations use a combination of User Semantics and Content-Based Algorithm.\n" \
                                                "\n If you want to see more recommendations you can use the cast, genre and movie inputs too.\n"
        if selected_genres:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected genre '{selected_genres}',\n" \
                                                    f"The movie recommendations presented here involve a dashboard that assesses films within the chosen genre, {selected_genres}, which the user has liked, reviewed, or highly rated in the past. Subsequently, the dashboard suggests movies akin to those preferences that the user has not yet watched. This recommendation system employs a combination of genre-based, user semantics, and content-based algorithms."
        if selected_cast:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected cast '{selected_cast}',\n" \
                                                    f"The movie recommendations provided below works in such a way that dashboard examines films featuring the chosen cast, cross-referencing them with the movies the user has already seen with those actors. The dashboard then suggests films the user has yet to watch, all of which star the selected actor"
        if selected_movie:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected movie '{selected_movie}',\n" \
                                                    f"The movie recommendation you get here is such that dashboard will give you movie-based recommendation for the selected movie,{selected_movie},  using the Content-Based Algorithm and KNN using Matrix Factorization based Algorithm. The movies recommended to you will be only the movies you have not watched."
    if not selected_user_id:
        explanation_text_1 = f"Hi New User, Welcome to Movie Recommendation dashboard.\n"

        if selected_genres:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected {selected_genres} as genre,\n" \
                                                    f"the movie recommendation you get below will be based on the genres you have selected. This works using a genre-based recommendation. Please click on 'Get Recommendations' to see the results"
        if selected_cast:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected {selected_cast} as cast,\n" \
                                                    f"the movie recommendation you get below will be based on the cast you have selected above.This works using a cast-based recommendation. Please click on 'Get Recommendations' to see the results"
        if selected_genres and selected_cast:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected genre {selected_genres} and cast {selected_cast},\n" \
                                                    f"The movie recommendation you get below will be based on the genres and the cast you have selected above. Recommendation will be based on both ethe options you selected.This works using a cast and genre-based recommendation. Please click on 'Get Recommendations' to see the results"
        if selected_movie:
            explanation_text = explanation_text_1 + f"\n\nAs you have selected movie {selected_movie}.\n" \
                                                    f"the movie recommendation you get here is movie-based recommendation using the Content-Based Algorithm and KNN using Matrix Factorization based Algorithm. This recommendation is using movie similarities and user ratings respectively into consideration. Please click on 'Get Recommendations' to see the results"

    return [html.P(explanation_text, style={'color': 'green', 'font-style': 'italic', 'font-weight': 'bold'})]


if __name__ == "__main__":
    app.run_server(debug=True)